# 決定係数の分析

## 訓練データ作成

### 正規化されたサンプル（＝質問）を用意


どのパラメータ（＝単語）が、分類結果（＝回答）に影響を与えるのかを見るには、パラメータごとの決定係数を比較します。

ただし、その前に訓練データが正規化されていないと、決定係数の比較ができないので、訓練データとなるTFベクターを正規化します。

#### TFベクターを、平均＝０、標準偏差＝１の配列へ変換します。

In [1]:
import numpy
from numpy import array
tf_vector_array = array([ # これは特徴語の出現回数リストです。
    [2, 0, 0, 0, 1],
    [3, 0, 0, 0, 2],
    [3, 0, 0, 1, 1],
    [0, 3, 0, 1, 0],
    [0, 2, 0, 1, 0],
    [0, 3, 0, 2, 0],
    [0, 0, 5, 1, 0],
    [0, 0, 3, 1, 0],
    [0, 0, 2, 1, 0],
    [0, 0, 4, 2, 0],
    ])
tf_vector_std_array = (tf_vector_array - tf_vector_array.mean(axis=0)) / tf_vector_array.std(axis=0)
tf_vector_std_array

array([[ 0.96076892, -0.64051262, -0.75482941, -1.58113883,  0.90453403],
       [ 1.76140969, -0.64051262, -0.75482941, -1.58113883,  2.41209076],
       [ 1.76140969, -0.64051262, -0.75482941,  0.        ,  0.90453403],
       [-0.64051262,  1.76140969, -0.75482941,  0.        , -0.60302269],
       [-0.64051262,  0.96076892, -0.75482941,  0.        , -0.60302269],
       [-0.64051262,  1.76140969, -0.75482941,  1.58113883, -0.60302269],
       [-0.64051262, -0.64051262,  1.94098992,  0.        , -0.60302269],
       [-0.64051262, -0.64051262,  0.86266219,  0.        , -0.60302269],
       [-0.64051262, -0.64051262,  0.32349832,  0.        , -0.60302269],
       [-0.64051262, -0.64051262,  1.40182605,  1.58113883, -0.60302269]])

#### 平均＝０になっているようです。

In [2]:
numpy.mean(tf_vector_std_array, axis=0)

array([ -8.88178420e-17,   4.44089210e-17,   0.00000000e+00,
         0.00000000e+00,  -8.88178420e-17])

#### 標準偏差＝１になっているようです。

In [3]:
numpy.std(tf_vector_std_array, axis=0)

array([ 1.,  1.,  1.,  1.,  1.])

#### 正規化されたベクターを、scikit-learn の引数として与えるために疎配列化します。

ここはいつも通りの手順になります

In [4]:
from scipy.sparse.csr import csr_matrix

tf_vector = csr_matrix(tf_vector_std_array) # これは正規化された特徴語の出現回数リストです。
tf_vector.toarray()

array([[ 0.96076892, -0.64051262, -0.75482941, -1.58113883,  0.90453403],
       [ 1.76140969, -0.64051262, -0.75482941, -1.58113883,  2.41209076],
       [ 1.76140969, -0.64051262, -0.75482941,  0.        ,  0.90453403],
       [-0.64051262,  1.76140969, -0.75482941,  0.        , -0.60302269],
       [-0.64051262,  0.96076892, -0.75482941,  0.        , -0.60302269],
       [-0.64051262,  1.76140969, -0.75482941,  1.58113883, -0.60302269],
       [-0.64051262, -0.64051262,  1.94098992,  0.        , -0.60302269],
       [-0.64051262, -0.64051262,  0.86266219,  0.        , -0.60302269],
       [-0.64051262, -0.64051262,  0.32349832,  0.        , -0.60302269],
       [-0.64051262, -0.64051262,  1.40182605,  1.58113883, -0.60302269]])

### クラス（回答）を用意

In [5]:
class_vector_array = array([
    1,
    1,
    1,
    2,
    2,
    2,
    3,
    3,
    3,
    3
    ])
class_vector_array # これは質問文ごとのクラスリスト（分類ラベルリスト）です。

array([1, 1, 1, 2, 2, 2, 3, 3, 3, 3])

## 学習実行

今回は動作確認のため、通常のパラメータで実行します。

In [6]:
from sklearn.linear_model import LogisticRegression

logisticRegression = LogisticRegression(
                        max_iter=100,
                        solver='newton-cg',
                        )
logisticRegression.fit(tf_vector, class_vector_array)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='newton-cg', tol=0.0001,
          verbose=0, warm_start=False)

## 決定係数の参照

学習の結果得られた決定係数をみてみます。

クラス（＝回答）ごとに、どの特徴（＝単語）の影響が高いかが一目でわかります。

In [7]:
logisticRegression.coef_

array([[ 0.88264775, -0.36588853, -0.42631075, -0.46715407,  0.71041679],
       [-0.3873536 ,  1.15251119, -0.67731546,  0.20197104, -0.33365974],
       [-0.52433643, -0.82158429,  1.03100829,  0.18867405, -0.44789634]])

#### クラス #0 の場合

１番目の特徴の影響が高い（値＝0.88でもっとも大きい）が、２番目の特徴の影響（0.7）もそこそこあることがわかります

In [8]:
logisticRegression.coef_[0]

array([ 0.88264775, -0.36588853, -0.42631075, -0.46715407,  0.71041679])

#### クラス #1 の場合

２番目の特徴の影響がかなり強い（値＝1.15でもっとも大きい）ことがわかります。

In [9]:
logisticRegression.coef_[1]

array([-0.3873536 ,  1.15251119, -0.67731546,  0.20197104, -0.33365974])